In [1]:
#Loading libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the merged dataset
merged_data = pd.read_csv("merged_data.csv")

In [3]:
#Feature Engineering#
# 1. Address Missing Data
# Numerical columns: Continuous and Count Data
numerical_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_columns:
    if merged_data[col].isnull().sum() > 0:
        # Use median for imputation
        merged_data[col].fillna(merged_data[col].median(), inplace=True)

# Categorical columns: Impute with the mode
categorical_columns = merged_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    if merged_data[col].isnull().sum() > 0:
        # Use mode for imputation
        merged_data[col].fillna(merged_data[col].mode()[0], inplace=True)

# Date columns: Forward-fill or backward-fill for imputation
date_columns = [col for col in merged_data.columns if 'Date' in col or 'date' in col]
for col in date_columns:
    if merged_data[col].isnull().sum() > 0:
        # Forward-fill method
        merged_data[col] = pd.to_datetime(merged_data[col], errors='coerce')
        merged_data[col].fillna(method='ffill', inplace=True)

In [7]:
# 2. One-Hot Encoding for Categorical Columns
# Verify available columns
print("Available columns:", merged_data.columns)

# Specify columns to encode (adjust if needed)
specified_categorical_columns = ['Product_Name', 'Company_Name', 'Address']

# Dynamically filter to include only existing columns
categorical_columns_to_encode = [
    col for col in specified_categorical_columns if col in merged_data.columns
]

if categorical_columns_to_encode:
    # Update: Correcting the OneHotEncoder usage by removing the 'sparse_output' argument
    # which is not valid. This behavior is controlled by 'sparse' argument. Switching accordingly.
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_features = pd.DataFrame(
        encoder.fit_transform(merged_data[categorical_columns_to_encode]),
        columns=encoder.get_feature_names_out(categorical_columns_to_encode)
    )

    # Add encoded features to the dataset
    merged_data = pd.concat([merged_data.reset_index(drop=True), encoded_features.reset_index(drop=True)], axis=1)

    # Drop original categorical columns after encoding
    merged_data = merged_data.drop(columns=categorical_columns_to_encode)
else:
    print("No valid categorical columns found for one-hot encoding.")

Available columns: Index(['Transaction_ID', 'Company_ID', 'Product_ID', 'Quantity',
       'Transaction_Date', 'Product_Price_x', 'Total_Cost', 'Product_Name',
       'Product_Price_y', 'Company_Name', 'Company_Profit', 'Address'],
      dtype='object')


In [9]:
# 3. Scale Numerical Features
# Define numerical columns for scaling (ensure they exist)
numerical_columns = [
    col for col in ['Product_Price', 'Quantity', 'Total_Cost', 'Company_Profit']
    if col in merged_data.columns
]

if numerical_columns:
    scaler = StandardScaler()
    merged_data[numerical_columns] = scaler.fit_transform(merged_data[numerical_columns])
else:
    print("No valid numerical columns found for scaling.")

# Final Check: Ensure all features are processed
print("Feature Engineering Complete. Dataset Preview:")
print(merged_data.head())

Feature Engineering Complete. Dataset Preview:
   Transaction_ID  Company_ID  Product_ID  Quantity Transaction_Date  \
0             1.0        88.0         6.0  0.076948       2024-03-26   
1             2.0        29.0        19.0  0.984139       2024-07-09   
2             3.0        28.0        18.0 -0.830243       2024-04-13   
3             4.0        85.0        12.0  0.258386       2023-09-06   
4             5.0        47.0         3.0 -0.467367       2021-07-06   

   Product_Price_x  Total_Cost  Product_Price_y  Company_Profit  \
0    194379.147964   -0.395485         179200.0       -0.013588   
1     97930.993380    0.013660          95200.0       -0.596943   
2    126095.547778   -0.551349         134400.0       -0.631825   
3    131600.000000   -0.473417          84000.0        1.550027   
4     99575.609634   -0.824112         100800.0       -1.881425   

   Product_Name_BudgetMaster Pro  ...  \
0                            0.0  ...   
1                            0.0  .

In [11]:
# Define X (features) and y (target)
X = merged_data.drop(columns=['Product_ID'])
y = merged_data['Product_ID']

# Convert target variable to categorical (if necessary)
y = y.astype(str)

### Split Data ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Fix: Remove or encode non-numerical columns in the features
# Drop non-numerical columns such as 'Transaction_Date' to avoid conversion errors
X_numerical = X.select_dtypes(include=['number'])

# Split Data (again) after correction
X_train, X_test, y_train, y_test = train_test_split(X_numerical, y, test_size=0.2, random_state=42)

# Model 1: Multinomial Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Multinomial Logistic Regression
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42, max_iter=1000)

# Train the model
logistic_model.fit(X_train, y_train)

# Predictions
y_pred_logistic = logistic_model.predict(X_test)

# Evaluation Metrics
logistic_metrics = {
    "Model": "Multinomial Logistic Regression",
    "Accuracy": accuracy_score(y_test, y_pred_logistic),
    "Classification Report": classification_report(y_test, y_pred_logistic, output_dict=True)
}

# Print results
print("Multinomial Logistic Regression Metrics:")
print(f"Accuracy: {logistic_metrics['Accuracy']}")
print(classification_report(y_test, y_pred_logistic))

# Save predictions to a DataFrame
logistic_predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_logistic})
logistic_predictions.to_csv("logistic_predictions.csv", index=False)

/root/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

In [17]:
#Model 2: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluation Metrics
rf_metrics = {
    "Model": "Random Forest Classifier",
    "Accuracy": accuracy_score(y_test, y_pred_rf),
    "Classification Report": classification_report(y_test, y_pred_rf, output_dict=True)
}

# Print results
print("Random Forest Classifier Metrics:")
print(f"Accuracy: {rf_metrics['Accuracy']}")
print(classification_report(y_test, y_pred_rf))

# Save predictions to a DataFrame
rf_predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred_rf})
rf_predictions.to_csv("rf_predictions.csv", index=False)

Random Forest Classifier Metrics:
Accuracy: 1.0
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        82
        10.0       1.00      1.00      1.00       266
        11.0       1.00      1.00      1.00        92
        12.0       1.00      1.00      1.00        97
        13.0       1.00      1.00      1.00        77
        14.0       1.00      1.00      1.00        93
        15.0       1.00      1.00      1.00        83
        16.0       1.00      1.00      1.00       106
        17.0       1.00      1.00      1.00        82
        18.0       1.00      1.00      1.00        96
        19.0       1.00      1.00      1.00        85
         2.0       1.00      1.00      1.00        88
        20.0       1.00      1.00      1.00        90
         3.0       1.00      1.00      1.00        98
         4.0       1.00      1.00      1.00        94
         5.0       1.00      1.00      1.00        97
         6.0       1.00      1.00

In [19]:
#Model Comparison Matrix
# Consolidate metrics into a single table
comparison_table = pd.DataFrame({
    "Model": [logistic_metrics["Model"], rf_metrics["Model"]],
    "Accuracy": [logistic_metrics["Accuracy"], rf_metrics["Accuracy"]],
    "Precision (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["precision"],
        rf_metrics["Classification Report"]["weighted avg"]["precision"]
    ],
    "Recall (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["recall"],
        rf_metrics["Classification Report"]["weighted avg"]["recall"]
    ],
    "F1-Score (Weighted Avg)": [
        logistic_metrics["Classification Report"]["weighted avg"]["f1-score"],
        rf_metrics["Classification Report"]["weighted avg"]["f1-score"]
    ]
})

# Save comparison table to CSV
comparison_table.to_csv("model_comparison.csv", index=False)

# Print the comparison table
print("Model Comparison Table:")
print(comparison_table)

Model Comparison Table:
                             Model  Accuracy  Precision (Weighted Avg)  \
0  Multinomial Logistic Regression    0.1465                  0.043627   
1         Random Forest Classifier    1.0000                  1.000000   

   Recall (Weighted Avg)  F1-Score (Weighted Avg)  
0                 0.1465                 0.054319  
1                 1.0000                 1.000000  


In [21]:
# Predict on the same sample data
sample_data = X_test.iloc[:12]  # Predicting 12 datapoints
logistic_predictions = logistic_model.predict(sample_data)
rf_predictions = rf_model.predict(sample_data)

# Combine predictions into a single DataFrame
comparison_df = pd.DataFrame({
    'Sample Index': sample_data.index,
    'Actual': y_test.loc[sample_data.index].values,
    'Logistic Regression Prediction': logistic_predictions,
    'Random Forest Prediction': rf_predictions
})

# Display the comparison table
print("\nComparison of Predictions Side by Side:")
print(comparison_df)


Comparison of Predictions Side by Side:
    Sample Index Actual Logistic Regression Prediction  \
0           6252   19.0                           10.0   
1           4684   10.0                           10.0   
2           1731   10.0                           10.0   
3           4742    3.0                           10.0   
4           4521    3.0                           10.0   
5           6340    9.0                           20.0   
6            576   17.0                           10.0   
7           5202    3.0                           10.0   
8           6363   14.0                           10.0   
9            439    9.0                           20.0   
10          2750    1.0                           10.0   
11          7487    3.0                           10.0   

   Random Forest Prediction  
0                      19.0  
1                      10.0  
2                      10.0  
3                       3.0  
4                       3.0  
5                       

In [25]:
from sklearn.metrics import accuracy_score, classification_report

# Correctly reference the RandomForestClassifier object defined previously
rf_pred_probs = rf_model.predict_proba(X_test)  # Changed 'random_forest_model' to 'rf_model'

# --- Printing the predictions side by side ---
# Create a DataFrame for the predictions to print them side by side
logistic_pred_df = pd.DataFrame(logistic_model.predict_proba(X_test), columns=[f'Product_{i}' for i in range(1, len(logistic_model.classes_) + 1)])  # Fixed logistic_pred_probs
rf_pred_df = pd.DataFrame(rf_pred_probs, columns=[f'Product_{i}' for i in range(1, len(rf_model.classes_) + 1)])

# Print the predicted probabilities from both models side by side
comparison_df = pd.concat([logistic_pred_df, rf_pred_df], axis=1, keys=["Logistic Regression", "Random Forest"])
print("\nPredicted Probabilities from Both Models (Logistic Regression vs Random Forest):")
print(comparison_df)

# --- Accuracy and Metrics ---
# Predict classes for evaluation (select the product with the highest probability)
logistic_preds = logistic_model.predict(X_test)
rf_preds = rf_model.predict(X_test)  # Changed 'random_forest_model' to 'rf_model'

# Calculate accuracy scores
logistic_accuracy = accuracy_score(y_test, logistic_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

print(f"\nLogistic Regression Accuracy: {logistic_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

# --- Display Classification Reports ---
print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, logistic_preds))

print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_preds))


Predicted Probabilities from Both Models (Logistic Regression vs Random Forest):
     Logistic Regression                                                    \
               Product_1 Product_2 Product_3 Product_4 Product_5 Product_6   
0               0.054202  0.092478  0.043681  0.041381  0.040047  0.045196   
1               0.057921  0.176506  0.061720  0.014107  0.047302  0.049999   
2               0.055280  0.212913  0.072135  0.010461  0.044329  0.054319   
3               0.055180  0.100047  0.050900  0.032042  0.044442  0.048006   
4               0.047289  0.125183  0.039666  0.054538  0.022790  0.047520   
...                  ...       ...       ...       ...       ...       ...   
1995            0.043210  0.089345  0.030660  0.075929  0.020228  0.040487   
1996            0.059453  0.152139  0.057209  0.011266  0.057753  0.042386   
1997            0.022788  0.176597  0.011589  0.135436  0.002767  0.028135   
1998            0.051655  0.054457  0.070244  0.008649  0.10

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a672b84-99aa-4a43-8d4f-014dc3397bc0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>